In [2]:
# !pip install -U deepeval
# !pip install -U bitsandbytes

In [7]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import BaseMetric
import pandas as pd
import numpy as np
import tqdm
import sqlite3
import os

In [9]:
path = '..'
os.chdir(path)
print(os.getcwd())

C:\Users\mvict\OneDrive\Desktop\Docs PPGI\Conteúdo\NLP\Trabalho 4


In [11]:
from custom_metrics.customMetric import customMetric

In [13]:
model_names = ['base', 'LR1e4', 'LR5e5']
scores = {key: [] for key in model_names}

for model_name in model_names: 
    
    # df_answer = pd.read_csv(f'answers/dev_queries_LLaMa-3-8b-{model_name}-clean.csv')
    df_answer = pd.read_csv(f'answers/dev_queries_LLaMa-3-8b-{model_name}.csv')
    
    for i in tqdm.tqdm(range(len(df_answer))):
        db_id = df_answer.iloc[i, 0]
        query = df_answer.iloc[i, 1]
        question = df_answer.iloc[i, 2]
        predicted_query = df_answer.iloc[i, 3]
    
        conn = sqlite3.connect(f"spider_data/spider_data/test_database/{db_id}/{db_id}.sqlite")
        conn.text_factory = bytes
        cursor = conn.cursor()
    
        try:
            cursor.execute(predicted_query)
            predicted_query_result =  cursor.fetchall()
        except:
            pass
    
        cursor.execute(query)
        query_result = cursor.fetchall()
    
        test_case = LLMTestCase(
            input= question,
            actual_output=predicted_query_result,
            expected_output=query_result
        )
    
        metric = customMetric()
        score = metric.measure(test_case)
    
        scores[model_name].append(score)
    
        del metric

100%|███████████████████████████████████████████████████████████████████████████████| 899/899 [00:06<00:00, 140.91it/s]


In [14]:
for model_name in model_names:
    final_score = round(sum(scores[model_name]) / len(scores[model_name]) * 100, 2)    
    print(f'The Execution Accuracy for model {model_name} is {final_score}%.')

The Execution Accuracy for model base is 41.27%.
The Execution Accuracy for model LR1e4 is 46.16%.
The Execution Accuracy for model LR5e5 is 55.17%.
